In [1]:
import os

In [2]:
%pwd

'c:\\Users\\gupta\\Documents\\GitHub\\Brain_Tumor_Detection\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\gupta\\Documents\\GitHub\\Brain_Tumor_Detection'

In [5]:
import tensorflow as tf

In [13]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [8]:
from CNN_Classifier.constants import *
from CNN_Classifier.utils.common import read_yaml, create_directories, save_json

In [9]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Brain_Tumor_Dataset",
            all_params=self.params,
            params_image_size=self.params.input_shape,
            params_batch_size=self.params.batch_size
        )
        return eval_config




In [10]:
from urllib.parse import urlparse

In [11]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            color_mode="grayscale",   
            shuffle=False,
            **dataflow_kwargs
        )

    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)

    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    

In [14]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()

except Exception as e:
   raise e

[2024-09-13 15:58:00,118: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-09-13 15:58:00,123: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-13 15:58:00,124: INFO: common: created directory at: artifacts]
Found 175 images belonging to 2 classes.


InvalidArgumentError: Graph execution error:

Detected at node 'model_1/normalization/sub' defined at (most recent call last):
    File "c:\Users\gupta\Documents\GitHub\Brain_Tumor_Detection\cnn_venv\lib\runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\gupta\Documents\GitHub\Brain_Tumor_Detection\cnn_venv\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\gupta\Documents\GitHub\Brain_Tumor_Detection\cnn_venv\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "c:\Users\gupta\Documents\GitHub\Brain_Tumor_Detection\cnn_venv\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "c:\Users\gupta\Documents\GitHub\Brain_Tumor_Detection\cnn_venv\lib\site-packages\ipykernel\kernelapp.py", line 739, in start
      self.io_loop.start()
    File "c:\Users\gupta\Documents\GitHub\Brain_Tumor_Detect